In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.mam.*
import jline.util.matrix.*
import kotlin.math.*

In [ ]:
val model = Network("model")

val M = 4
// Create 4 single-server queues with FCFS scheduling
val nodes = Array<Node?>(4) { null }
for (i in 0 until M) {
    nodes[i] = Queue(model, "Queue${i+1}", SchedStrategy.FCFS)
    // Single server by default (numberOfServers = 1)
}

val source = Source(model, "Source")
val sink = Sink(model, "Sink")

// Create job classes
val closedClass = ClosedClass(model, "ClosedClass", 100, nodes[0], 0)  // High population
val openClass = OpenClass(model, "OpenClass", 0)

// Set service rates
for (i in 0 until M) {
    nodes[i]!!.setService(closedClass, Exp((i + 1).toDouble()))
    nodes[i]!!.setService(openClass, Exp(sqrt((i + 1).toDouble())))
}

// Set arrival rate for open class using APH distribution
source.setArrival(openClass, APH.fitMeanAndSCV(3.0, 64.0))

println("Created 4 single-server FCFS queues with service rates 1, 2, 3, 4")
println("Closed class: 100 jobs, Open class arrivals: APH(mean=3, SCV=64)")

In [ ]:
// Setup routing matrix
val numStations = model.numberOfStations
val numClasses = model.numberOfClasses

val P = model.initRoutingMatrix()

// Closed class routing: Queue1 -> Queue2 -> Queue3 -> Queue4 -> Queue1 (circular)
P.set(closedClass, closedClass, nodes[0], nodes[1], 1.0)
P.set(closedClass, closedClass, nodes[1], nodes[2], 1.0)
P.set(closedClass, closedClass, nodes[2], nodes[3], 1.0)
P.set(closedClass, closedClass, nodes[3], nodes[0], 1.0)

// Open class routing: Source -> Queue1 -> Queue2 -> Queue3 -> Sink (excludes Queue4)
P.set(openClass, openClass, source, nodes[0], 1.0)
P.set(openClass, openClass, nodes[0], nodes[1], 1.0)
P.set(openClass, openClass, nodes[1], nodes[2], 1.0)
P.set(openClass, openClass, nodes[2], sink, 1.0)

model.link(P)

println("\nRouting configured:")
println("  Closed class: Queue1->Queue2->Queue3->Queue4->Queue1 (circular)")
println("  Open class: Source->Queue1->Queue2->Queue3->Sink (excludes Queue4)")

In [ ]:
println("\nThis example shows the execution of the solver on a 2-class mixed model with 4 single server nodes.")

// Default options matching MATLAB
val seed = 23000
val cutoff = 3
val verbose = true
val keep = false
val samples = 20000  // 2e4 in MATLAB

// Create solvers with appropriate configurations
// Note: CTMC and SSA are commented out in MATLAB due to high population
val solvers = mutableListOf<NetworkSolver>()

// CTMC is commented out in MATLAB (infinite on this model)
// try {
//     solvers.add(CTMC(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))
// } catch (e: Exception) {
//     println("CTMC solver not available: ${e.message}")
// }

try {
    solvers.add(JMT(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed, "samples", samples))
} catch (e: Exception) {
    println("JMT solver not available: ${e.message}")
}

// SSA is commented out in MATLAB
// try {
//     solvers.add(SSA(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed, "samples", samples))
// } catch (e: Exception) {
//     println("SSA solver not available: ${e.message}")
// }

// FLD is very slow with APH arrivals in mixed networks due to ODE stiffness
// try {
//     solvers.add(FLD(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))
// } catch (e: Exception) {
//     println("Fluid solver not available: ${e.message}")
// }

try {
    solvers.add(MVA(model, "method", "lin", "keep", keep, "verbose", verbose, "seed", seed))
} catch (e: Exception) {
    println("MVA solver not available: ${e.message}")
}

try {
    solvers.add(NC(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))
} catch (e: Exception) {
    println("NC solver not available: ${e.message}")
}

try {
    solvers.add(MAM(model, "cutoff", cutoff, "keep", keep, "verbose", verbose, "seed", seed))
} catch (e: Exception) {
    println("MAM solver not available: ${e.message}")
}

println("\nAvailable solvers: ${solvers.size}")
println("Note: CTMC, SSA, and FLD omitted due to high population/performance")

In [ ]:
// Solve and display resultssolvers.forEachIndexed { index, solver ->    try {        println("\n${"+".repeat(50)}")                println("${"+".repeat(50)}")                val avgTable = solver.avgTable        avgTable.print()            } catch (e: Exception) {        println("\nSOLVER ${solver.name.replace("Solver", "")} FAILED: ${e.message}")    }}

## Results Analysis

This high-congestion mixed queueing network demonstrates:

1. **High Population Effects**: 100 closed class jobs create significant congestion
2. **FCFS Discipline**: First-come-first-served creates potential for large queue buildups
3. **APH Arrivals**: High variability arrivals (SCV=64) for open class
4. **Solver Limitations**: Exact methods (CTMC/SSA) infeasible for high populations

**Expected Behavior:**
- **Queue1**: Likely to have highest queue length due to being bottleneck (rate=1)
- **Response Times**: Very high for closed class due to congestion
- **Open Class**: May experience blocking or delays due to closed class presence

**Solver Comparison:**
- **JMT**: Simulation-based, should provide accurate estimates
- **MVA**: Linearization method for handling high populations
- **NC**: Normalizing constant method with approximations
- **MAM**: Matrix-analytic method for mixed models